# Statistics

During the last few days, we learned fundamental programming.
While this knowledge is useful, it usually needs to be paired with some basic knowledge about mathematics.
Therefore, we will now investigate our signals a little bit deeper.

## Table of contents

1. [Inter spike interval](#Inter-spike-interval)
2. [Simulation](#Simulation)
3. [Compare model to reality](#Compare-model-to-reality)
4. [Distributions](#Distributions)
    1. [Normal distribution](#Normal-distribution)
    2. [Mode](#Mode)
    3. [Median](#Median)
    4. [Gamma distribution](#Gamma-distribution)
5. [Adapting our simulation code](#Adapting-our-simulation-code)
6. [Fitting](#Fitting)
7. [Numerical Integration](#Numerical-Integration)
8. [Numerical differentiation](#Numerical-differentiation)
9. [Rubber duck debugging](#Rubber-duck-debugging)

## Inter spike interval

Between two spikes a neuro needs time to recover.
This [refractory period](https://en.wikipedia.org/wiki/Refractory_period_(physiology)) has a minimal length.
The units we showed you were filtered with multiple methods.
One of them was the minimal refractory period.
We will now investigate it.

## Simulation

Considering that we have now "incorrect unit" we have to create one, but how?
The answer to this is simulation.
Especially in Physics simulations are often used tool to answer questions that cannot be answered with simple experiments.
If we want to know how galaxies form we can neither make one in our own backyard nor can we observe it in our lifetimes,
so we build a mathematical model in a computer and investigate it.

In biology, computer simulations are more difficult to perform, because we lack a sufficiently advanced mathematical understanding of the problems we investigate.
Expressed in a simpler way “it is easier to calculate how two galaxies collide, than how two cell interacts with each other.
You can see this in the way we "simulated" the our two neurons.

We take a starting point and then roll a random number afterwards.
Adding the random number to the last time gives us the new spike time.
Thereby we have a randomly firing neuron.

We cover simulations, because I believe that during your career you will may encounter questions that can be answered by writing a short program and running it instead of using a plant or animal and that the use of simulation will slowly proliferate within biology.
For the latter case always remember that a simulation is a simplified mathematical model and therefore flawed, so if you use it always ask which corners were cut and how this will influence your research.

So if we now simulate a neurin our code could look like this:

```python
import random

def create_random_neuron(start_time, minimal_delay, maximal_delay, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param minimal_delay the minimal distance between spikes (refractory period)
    @param maximal_delay the maximal distance between spikes (no biologial meaning)
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = random.uniform(minimal_delay, maximal_delay)
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

# To ensure that the random module produces repeatable results we have to seed it
# This makes sure if we run the same algortihm on two machines the results are equal
# If you do not specify it a seed is chosen autoamtically, usually the system time
random.seed(42)
print(create_random_neuron(0, 0.2, 5.0, 100))
```

Please use the code above to create a neuron.
Then calculate the time difference between the spikes (inter-spike-interval) and visualize the results.

In [ ]:
# Your code should be added here

<details>
<summary> Show suggested solution </summary>

```Python
import random
import matplotlib.pyplot

def create_random_neuron(start_time, minimal_delay, maximal_delay, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param minimal_delay the minimal distance between spikes (refractory period)
    @param maximal_delay the maximal distance between spikes (no biologial meaning)
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = random.uniform(minimal_delay, maximal_delay)
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

# To ensure that the random module produces repeatable results we have to seed it
# This makes sure if we run the same algortihm on two machines the results are equal
# If you do not specify it a seed is chosen autoamtically, usually the system time
random.seed(42)
random_spikes = create_random_neuron(0, 0.2, 5.0, 100)
interval_random_spikes = get_inter_spike_intervals(random_spikes)

matplotlib.pyplot.hist(interval_random_spikes)
matplotlib.pyplot.show()
```

</details>

## Compare model to reality

You have seen a rough distribution, but to correctly fine tune your model you should compare it to reality.
Please read in the contents of ```./data_neuron/session_2023111501010_units.csv``` as we did last time and plot the inter-spike-interval of it.

In [ ]:
# Your code should be added here

<details>
<summary> Show suggested solution </summary>

```Python
import random
import csv
import pathlib
import matplotlib.pyplot

def create_random_neuron(start_time, minimal_delay, maximal_delay, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param minimal_delay the minimal distance between spikes (refractory period)
    @param maximal_delay the maximal distance between spikes (no biologial meaning)
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = random.uniform(minimal_delay, maximal_delay)
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

spikes_times_units = get_spike_times_for_all_units(units_file_path)
max_duration = 0
for unit_id in spikes_times_units.keys():
    max_time_unit = max(spikes_times_units[unit_id])
    max_duration = max(max_time_unit, max_duration)

for unit in spikes_times_units.keys():
    matplotlib.pyplot.title(f"Unit {unit}")   
    matplotlib.pyplot
    matplotlib.pyplot.hist(get_inter_spike_intervals(spikes_times_units[unit]))
    matplotlib.pyplot.xlabel("Interval in seconds")
    matplotlib.pyplot.ylabel("Number of spikes")
    matplotlib.pyplot.show()

random.seed(42)
random_spikes = create_random_neuron(0, 0.2, 5.0, max_duration)
interval_random_spikes = get_inter_spike_intervals(random_spikes)

matplotlib.pyplot.title(f"Simulation")
matplotlib.pyplot.hist(interval_random_spikes)
matplotlib.pyplot.xlabel("Interval in seconds")
matplotlib.pyplot.ylabel("Number of spikes")
matplotlib.pyplot.show()
```

</details>

In [ ]:
import random
import csv
import pathlib
import matplotlib.pyplot

def create_random_neuron(start_time, minimal_delay, maximal_delay, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param minimal_delay the minimal distance between spikes (refractory period)
    @param maximal_delay the maximal distance between spikes (no biologial meaning)
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = random.uniform(minimal_delay, maximal_delay)
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

spikes_times_units = get_spike_times_for_all_units(units_file_path)
max_duration = 0
for unit_id in spikes_times_units.keys():
    max_time_unit = max(spikes_times_units[unit_id])
    max_duration = max(max_time_unit, max_duration)

for unit in spikes_times_units.keys():
    matplotlib.pyplot.title(f"Unit {unit}")   
    matplotlib.pyplot
    matplotlib.pyplot.hist(get_inter_spike_intervals(spikes_times_units[unit]))
    matplotlib.pyplot.xlabel("Interval in seconds")
    matplotlib.pyplot.ylabel("Number of spikes")
    matplotlib.pyplot.show()

random.seed(42)
random_spikes = create_random_neuron(0, 0.2, 5.0, max_duration)
interval_random_spikes = get_inter_spike_intervals(random_spikes)

matplotlib.pyplot.title(f"Simulation")
matplotlib.pyplot.hist(interval_random_spikes)
matplotlib.pyplot.xlabel("Interval in seconds")
matplotlib.pyplot.ylabel("Number of spikes")
matplotlib.pyplot.show()

## Distributions

As you have observed our simulated neuron does not look like any of the units we measured at all.
To answer the question why we have to do a little bit more statisic.
In our simulation we used ```random.uniform```, this means all numbers between lower and upper bound had the same chance to be drawn,
so our interrvals were equally distributed.

The real data show a bias towards lower numbers however.
So they are not equally distributed.
This means we need to look at different distributions, so we can correctly simulate a neuron.

### Normal distribution

The most often used distribution is the [normal or gaussian distribution](https://en.wikipedia.org/wiki/Normal_distribution).
It is biased towards a center point which is also its average or [mean](https://en.wikipedia.org/wiki/Mean#Mean_of_a_probability_distribution).
It spreads around its mean, ths size of the spread is described by the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation).
The standard deviation can be used how probable values are.
The further away the less likely we are to draw the value.
The chance to draw from an interval and the corresponding standard deviation ($\sigma$) is visualized in the following figure from
[Wikipedia](https://commons.wikimedia.org/wiki/File:Standard_deviation_diagram.svg):

![A bell shaped normal distribution](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Standard_deviation_diagram.svg/640px-Standard_deviation_diagram.svg.png)

If we add up the numbers we find out that 68.2 % of all values are within $1 \sigma $ of the mean.

Now why do we care?
The answer is the [central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem).
Simplified it states: "If you combine a large number of independent random variables, the resulting distribution will be the normal distribution."
Since a lot of things are independent random variables this distribution is very common.
So if you throw a dice often enough the sum of the eyes is a normally distributed,
if you measure the blood pressure of people you expect it to be normally distributed.

To get acquainted with the normal distribution please create one with ```random.gauss``` and use ```numpy.mean``` and ```numpy.std``` to recover the mean and standard deviation values you set.
Please interpret the results.

In [ ]:
# Add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import random
import numpy

# Higher sample numbers lead to a better approximation
number_of_samples = 10000
# Chose an arbitrary number here
random.seed(12)

# The standard deviation has to be bigger than 0
# otherwise you can chose any number you want
mean = 2.5
standard_deviation = 3.0

values = list()
for index in range(0, number_of_samples):
    values.append(random.gauss(mean, standard_deviation))

print(f"Mean given: {mean}, calculated: {numpy.mean(values)}")
print(f"Standard deviation given:{standard_deviation}, calculated {numpy.std(values)}")
```

Running the code we fail to recover mean and standard deviation.
This lies in the nature of random variables,
they approach the values of the distribution with higher sample numbers,
but will only reproduce them if the sample size reaches infinity.
</details>

### Mode

If we compare the normal distribution to the data we obtained we may find it an unsatisfying approximation.
Something does not fit, most of the values should shift to the left,
in other words the [mode](https://en.wikipedia.org/wiki/Mode_(statistics)) is wrong.

The mode is the most common value within a distribution, in other words its "peak".
So we need to search for a distribution in which the mode differs from the mean.

### Median

Before we talk about this better distribution lets us talk about another useful statistical measure:
the [median](https://en.wikipedia.org/wiki/Median) or the value in the middle.

We often use it to overcome the sensibility of the mean to outliers.
Let me give you a simple example based on a list of buckets:

| Bucket | Content |
| ------ | ------- |
| 1      | 0.4     |
| 2      | 0.5     |
| 3      | 0.6     |
| 4      | 0.5     |
| 5      | 8.0     |

If you ask how much content do I get if I combine them the average of 1.0 l might be helpful,
but if you get one at random and ask how much content should you expect the median of 0.5 l might be the better measurement.
It is often more useful for very assymetric distributions like [wealth](https://en.wikipedia.org/wiki/Economic_inequality),
as shown in the following figure from [wikipedia](https://en.wikipedia.org/wiki/File:Global_Wealth_Distribution_2020_(Property).svg).

![A figure showing that a small fraction of the worlds population owns most of the world material wealth](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Global_Wealth_Distribution_2020_%28Property%29.svg/444px-Global_Wealth_Distribution_2020_%28Property%29.svg.png)

### Gamma distribution

Now thatwe have learned more about statistics let us search for a distribution,
that is similar to the normal distribution but has a shifted mode.
The first choice would be the [gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution),
which is the continious equivalent to the [poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution).
To better visualize it lets look at this figure from [wikipedia](https://en.wikipedia.org/wiki/File:Gammapdf252.svg).

![Multiple examples of the gamma distribution](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Gammapdf252.svg/384px-Gammapdf252.svg.png)

As you can see we have a parameter $\alpha$ influencing the position of the mode within all values
and a parameter $\beta$ influencing the spread of the values.

We can draw from this distribution using ```random.gammavariate```.
Please draw a few samples and visualize the result.

In [ ]:
# Add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import random
import matplotlib.pyplot

# Higher sample numbers lead to a better approximation
number_of_samples = 10000
# Chose an arbitrary number here
random.seed(15)

# The standard deviation has to be bigger than 0
# otherwise you can chose any number you want
shape = 3
width = 5

values = list()
for index in range(0, number_of_samples):
    values.append(random.gammavariate(shape, scale))

matplotlib.pyplot.hist(values)
matplotlib.pyplot.show()
```
</details>

## Adapting our simulation code

Now that we have found a fitting function to describe our neurons,
we can try to add it into code.
Instead of drawing from a uniform distribution we now want to draw from an arbitrary distribution,
so we give  ```def create_random_neuron``` a **function** as an argument.
From this **function** we then get a distance,
so our code would look like this:

```Python
def create_random_neuron(start_time, get_distance, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param get_distance a function returning a distance
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = get_distance()
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times
```

We would then supply the **function** by creating a wrapper filling in the arguments:

```Python
# Example values
shape = 2.0
width = 3.0

def get_distance():
    return random.gammavariant(shape, width)

create_random_neuron(0, get_distance, 40)
```

This is a little bit inconvenient, because we would have to create one global **variable** for every **function** we generate,
so we have to find a more elegant solution.
So we write a **function** generating a **function** with these paremeters:

```Python
def get_gamma(shape, width):
    def sample_gamma():
        return random.gammavariant(shape, width)
    return sample_gamma

create_random_neuron(0, get_gamma(2.0, 3.0), 40)
```

There is of coures a way to write this shorter using [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions).
Lambda expressions are short nameless functions.
The begin with the keyword ```lambda``` followed by the list of arguments and a ```:```.
After the ```:``` the return value follows.
Here is a short example:

```Python
def square_(x)
    return x**2
a = square

# Now the same as a lambda expression
a = lablda x: x**2
````

Using lambda expressions our code would the look like this:

```Python
create_random_neuron(0, lambda : random.gammavariant(shape, width), 40)
```

To get acquainted with **functions** as arguments please simulate a neuron once with a gaussian and then a gamma distribution.
Use lambda expressions for one of them.
Please visualize the inter-spike-invervals.

In [ ]:
# Add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import random
import matplotlib.pyplot

def create_random_neuron(start_time, get_distance, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param get_distance a function returning a distance
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    last_time = start_time
    while True:
        time_difference = get_distance()
        # Since both our distirbutions can generate negative values,
        # we should redraw until we get positive ones
        while time_difference <= 0:
            time_difference = get_distance()
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

def get_gaussian(mean, std):
    def sample_gaussian():
        return random.gauss(mean, std)
    return sample_gaussian

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

random.seed(42)

# Finding good parameters here may take a while
gauss_values = create_random_neuron(0, get_gaussian(0.7, 0.2), 500)
gamma_values = create_random_neuron(0, lambda: random.gauss(0.3, 0.7), 500)

matplotlib.pyplot.title("Gauss")
matplotlib.pyplot.hist(get_inter_spike_intervals(gauss_values))
matplotlib.pyplot.show()

matplotlib.pyplot.title("Gamma")
matplotlib.pyplot.hist(get_inter_spike_intervals(gamma_values))
matplotlib.pyplot.show()
```
</details>

## Fitting

Now that we have found a fitting function to describe our unit,
we stil lack the correct parameter.
What are ```shape``` and ```width``` for our units?

To answer this we can fit a function to our data.
Fitting in itself is not a easy topic, especially if we have higher dimensional data.
Instead of doing a side lecture we will use ```scipy.optimize.curve_fit``` and hope for the best.
As an example we will fit a gaussian to our dat.

Since this is not the easiest function we will use it as an exmaple an exercise on a polynomial later.

To fit a function we first have to create it.
```scipy.optimize.curve_fit```, will replace the first value in it and optimize the rest.
So we write the function as a Python **function**:

```Python
def gauss(x, mean, std, scale):
    divisor = numpy.sqrt(2 * numpy.pi * (std**2))
    exponent = -((x - mean)**2) / (2 * (std**2)) 
    return numpy.exp(exponent) /  divisor
```

We then get the values to fit.
We need x-values and corresponding y-values for the curve fit.
We will obtain those by using ```numpy.histogram```.
Considering that the normal distribution is a probability distirbution we have to normalize the bins-sizes by the total sample size:

```
y_values = bin_values / len(inter_spike_intervals)
```

Finally will call curve fit and investigate the results.

Please execute the cell below and interpret the results.

In [ ]:
import numpy
import scipy
import pathlib
import csv
import matplotlib.pyplot

def gauss(x, mean, std):
    divisor = numpy.sqrt(2 * numpy.pi * (std**2))
    exponent = -((x - mean)**2) / (2 * (std**2)) 
    return numpy.exp(exponent) /  divisor
    

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

# We chose unit 17 here because it is rather broad
inter_spike_intervals = get_inter_spike_intervals(get_spike_times_for_all_units(units_file_path)[17])

bin_values, bin_edges = numpy.histogram(inter_spike_intervals, bins = 500)
y_values = bin_values / len(inter_spike_intervals)
# Average the bin edges into bin values
x_values = bin_edges[:-1] + bin_edges[1] / 2

parameters, covariance = scipy.optimize.curve_fit(gauss, x_values, y_values)

matplotlib.pyplot.plot(x_values, gauss(x_values, *parameters))
matplotlib.pyplot.plot(x_values, y_values)
matplotlib.pyplot.show()

The gamma-distribution is given in the following form according to its [documentation](https://docs.python.org/3/library/random.html#real-valued-distributions):
$$       
\gamma(x) =  \frac{x ^{\alpha - 1}  e^{-x / \beta}}{\Gamma(alpha)  \beta^{\alpha}}
$$

This can not be fitted directly, instead we will have to use a specialized function for it.
If you encounter functions that are difficutl to describe or can not easily fit,
consider fitting a [polynomial](https://en.wikipedia.org/wiki/Polynomial).
They often yield good enoug results.
Pleas use the cell below to fit a polynomial of 5 degree.

In [ ]:
# Add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import numpy
import scipy
import pathlib
import csv
import matplotlib.pyplot

def polynomial(x, a0, a1, a2, a3, a4, a5):
    return a0 + x*a1 + x**2*a2 + x**3*a3 + x**4*a4 + x**5*a5

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

# We chose unit 17 here because it is rather broad
inter_spike_intervals = get_inter_spike_intervals(get_spike_times_for_all_units(units_file_path)[17])

bin_values, bin_edges = numpy.histogram(inter_spike_intervals, bins = 500)
y_values = bin_values / len(inter_spike_intervals)
# Average the bin edges into bin values
x_values = bin_edges[:-1] + bin_edges[1] / 2

parameters, covariance = scipy.optimize.curve_fit(polynomial, x_values, y_values)

matplotlib.pyplot.plot(x_values, polynomial(x_values, *parameters))
matplotlib.pyplot.plot(x_values, y_values)
matplotlib.pyplot.show()
```
</details>

As mentioned previously the polynomial does not fit our distribution well,
so we have will now fit the distirbution directly.
Pleas consult the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html) of ```scipy.stats.gamma```
especially the fit **method** and use it to fit our data.
Then display the probability density function (pdf).
Forcefully normalize it to one, so it can be compared to our data:
```fit_y_values /= numpy.sum(fit_y_values)```

In [ ]:
# Add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import matplotlib.pyplot
import numpy
import scipy

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

inter_spike_intervals_unit = get_inter_spike_intervals(get_spike_times_for_all_units(units_file_path)[17])

values, bin_edges = numpy.histogram(inter_spike_intervals_unit, bins = 500)
y_values = values / len(inter_spike_intervals_unit)
# Average the bin edges into bin values
x_values = bin_edges[:-1] + bin_edges[1] / 2

fitted_values = scipy.stats.gamma.fit(inter_spike_intervals_unit)

# Scipy produces a none normalize probabilty density function, which we have to correct
fit_y_values = scipy.stats.gamma.pdf(x_values, *fitted_values)
fit_y_values /= numpy.sum(fit_y_values)

matplotlib.pyplot.plot(x_values, fit_y_values)
matplotlib.pyplot.plot(x_values, y_values)
matplotlib.pyplot.show()
```
</details>

## Numerical Integration

As you see our plots are quite similar, but why did we have to normalize?
The answer is that the pdf is the probability-*density*-function, so it describes the density of the probability.
To get the real probability we have to integrate it.

Since we have a computer at our hands we will integrate it [numericaly](https://en.wikipedia.org/wiki/Numerical_integration).
You may recall from yourn studies that an [integral](https://en.wikipedia.org/wiki/Integral) is the continious analogue to a sum,
so numerical integration just means we sum things up.

We will use the [quadrature rule based on step functions](https://en.wikipedia.org/wiki/Numerical_integration#Quadrature_rules_based_on_step_functions).
This means we will integrate at a point by forming a rectangle around it.
The width of the rectangle is defined by our step-size, which we have to choose corrctly.
The height is the center value of the step.
We can either obtain it by broadening one value or averaging between two values.
Here is an illsutration from [wikipedia](https://commons.wikimedia.org/wiki/File:Integration_rectangle.svg) demonstrating this integration:

![A graph showing boxes on a curve](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Integration_rectangle.svg/640px-Integration_rectangle.svg.png)

Here is a short example integrating a polynomial with both methods:

```Python
import matplotlib.pyplot

def poylnomial(x):
    return x**2

def integrate_one_value(begin, end, function, step_size):
    y_values = list()
    current_value = 0
    for x_value in range(begin, end + 1, step_size):
        current_value += function(x_value) * step_size
        y_values.append(current_value)
    return y_values

def integrate_average(begin, end, function, step_size):
    y_values = list()
    current_value = 0
    for x_value in range(begin, end + 1, step_size):
        half_step = step_size / 2
        average_height = (function(x_value - half_step) + function(x_value + half_step)) / 2
        current_value += average_height  * step_size
        y_values.append(current_value)
    return y_values

begin = 0
end = 100
step_size = 1
x_values = range(begin, end + 1, step_size)

y_values_one = integrate_one_value(begin, end, poylnomial, step_size)
y_values_average = integrate_average(begin, end, poylnomial, step_size)
y_values_analytic = [x**3/3 for x in x_values]

matplotlib.pyplot.plot(x_values, y_values_one, label = "One value")
matplotlib.pyplot.plot(x_values, y_values_average, label = "Average")
matplotlib.pyplot.plot(x_values, y_values_analytic, label = "Analytic")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()
```

Please use the knowledge you gained to integrate the cosinus.
To get better results you may wish to use ```numyp.arange``` and use smaller step_sizes.
Liming the end of the integration to a low multiple of $\pi$ using ```numpy.pi``` might also be helpful.

In [ ]:
# Please add your code here

<details>
<summary> Show suggested solution </summary>

```Python
import matplotlib.pyplot
import numpy

def integrate_one_value(begin, end, function, step_size):
    y_values = list()
    current_value = 0
    for x_value in numpy.arange(begin, end + 1, step_size):
        current_value += function(x_value) * step_size
        y_values.append(current_value)
    return y_values

def integrate_average(begin, end, function, step_size):
    y_values = list()
    current_value = 0
    for x_value in numpy.arange(begin, end + 1, step_size):
        half_step = step_size / 2
        average_height = (function(x_value - half_step) + function(x_value + half_step)) / 2
        current_value += average_height  * step_size
        y_values.append(current_value)
    return y_values

begin = 0
end = 8*numpy.pi
step_size = end/300
x_values = numpy.arange(begin, end + 1, step_size)

y_values_one = integrate_one_value(begin, end, numpy.cos, step_size)
y_values_average = integrate_average(begin, end, numpy.cos, step_size)
y_values_analytic = [numpy.sin(x) for x in x_values]

matplotlib.pyplot.plot(x_values, y_values_one, label = "One value")
matplotlib.pyplot.plot(x_values, y_values_average, label = "Average")
matplotlib.pyplot.plot(x_values, y_values_analytic, label = "Analytic")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()
```
</details>

We can now finally finish testing our fit.
Using the one value method we mutliply all the y-values of the pdf with a step-size and obtain the probability belonging to every x-value.
The step size is given as the distance between the x-values, because we need to sample the distribtuion fully.
Below you find the final code to investigate our fit.

In [ ]:
import matplotlib.pyplot
import numpy
import scipy

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

inter_spike_intervals_unit = get_inter_spike_intervals(get_spike_times_for_all_units(units_file_path)[17])

values, bin_edges = numpy.histogram(inter_spike_intervals_unit, bins = 500)
y_values = values / len(inter_spike_intervals_unit)
# Average the bin edges into bin values
x_values = (bin_edges[:-1] + bin_edges[1]) / 2

fitted_values = scipy.stats.gamma.fit(inter_spike_intervals_unit)

# Scipy produces a none normalize probabilty density function, which we have to correct through integration
# Using the step_distance we can turn our density into a probability
step_distance = x_values[1] - x_values[0]
fit_y_values = scipy.stats.gamma.pdf(x_values, *fitted_values) * step_distance

matplotlib.pyplot.plot(x_values, fit_y_values, label= "Fit")
matplotlib.pyplot.plot(x_values, y_values, label = "Data")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()

We now have the parameters to finish our simulation and finish our neuron.
We can now use ```scipy.stats.gamma.rvs``` with the fit parameters to simulate the neuron an plot it
as shown in the cell below.
Please execute it and enjoy the results.

In [ ]:
import pathlib
import matplotlib.pyplot
import csv
import numpy
import scipy

def create_random_neuron(start_time, get_distance, end_recording):
    """!
    @brief Creates random neuron data
    @details This creates a uniformly disributed neuron signal

    @param start_time the beginning of the firing
    @param get_distance a function returning a distance
    @param end_recording the largest permitted spike time
    @return a list with spike times
    """
    spike_times = list()
    # Note that the start time is not a spike otherwise the results would not be random enough
    last_time = start_time
    while True: # This is Pythons version of a do while loop: https://en.wikipedia.org/wiki/Do_while_loop
        time_difference = get_distance()
        last_time += time_difference
        if last_time < end_recording:
            spike_times.append(last_time)
        else:
            break
    return spike_times

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
    return differences

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

inter_spike_intervals_unit = get_inter_spike_intervals(get_spike_times_for_all_units(units_file_path)[17])

values, bin_edges = numpy.histogram(inter_spike_intervals_unit, bins = 500)
y_values = values / len(inter_spike_intervals_unit)
# Average the bin edges into bin values
x_values = (bin_edges[:-1] + bin_edges[1]) / 2

fitted_values = scipy.stats.gamma.fit(inter_spike_intervals_unit)

# Scipy produces a none normalize probabilty density function, which we have to correct
step_distance = x_values[1] - x_values[0]
fit_y_values = scipy.stats.gamma.pdf(x_values, *fitted_values) * step_distance

# Scipy uses numpy random instead of random, so we have to seed it instead
numpy.random.seed(42)
random_spikes = create_random_neuron(0, lambda:scipy.stats.gamma.rvs(*fitted_values), 3400)
interval_random_spikes = get_inter_spike_intervals(random_spikes)

matplotlib.pyplot.title(f"Simulation")
matplotlib.pyplot.hist(interval_random_spikes)
matplotlib.pyplot.xlabel("Interval in seconds")
matplotlib.pyplot.ylabel("Number of spikes")
matplotlib.pyplot.show()

## Numerical differentiation

Now that we have succesfully simulated and integrated there is one topic left:
[numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation).
It is the counter part to numerical integration and we can use it to get a derivative.

During my high-school time the derivative was introduced via the [slope](https://en.wikipedia.org/wiki/Slope) of a function.
We counted how much we went up or down on one axis comparted to the other,
as shown in the following [illustration from Wikipedia](https://commons.wikimedia.org/wiki/File:Wiki_slope_in_2d.svg):

![A slope illustrated with the interval steps on the x- and y-axis](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Wiki_slope_in_2d.svg/445px-Wiki_slope_in_2d.svg.png)

Luckily, for us [numerical differentiation](https://en.wikipedia.org/wiki/Numerical_differentiation) did not progress much further, so we get the first derivative with the the original formula:

$$
m = \frac{y_{2} - y_{1}}{x_{2} - x_{1}}
$$

This gives us the forward slope, an approximation for the first order derivative.
We can now abstract the formula by introducing the stepsize $h$.
Our formula now takes the following form:

$$
f^{'}(x) = \frac{f(x + h) - f(x)}{h}
$$

If you need a symmetric derivative, you may choose to look around the point of interest and use:

$$
f^{'}(x) = \frac{f(x + h) - f(x - h)}{2 h}
$$

Should you need to calculate derivatives of mathematical functions you should look up [Taylor expansion](https://en.wikipedia.org/wiki/Taylor_series) and [finite difference coefficients](https://en.wikipedia.org/wiki/Finite_difference_coefficient).
In this case you should also pay attention to the size of h.

As a first exercise please calculate and visualize the numerical derivative of $x^{3}$ betwen 0 and 10 in the next cell:

In [ ]:
# Write your code here

<details>
<summary> Show suggested solution </summary>

```Python
import matplotlib.pyplot
import numpy

def get_forward_derivative(series_x, series_y):
    if len(series_x) != len(series_y):
        raise ValueError(f"{series_x} did not have the same number of elements as {series_y}")
    return_x = list()
    return_y = list()
    # Remeber that the last element has no sucessor and we can therefore not get a derivative for it
    for index in range(0, len(series_x) - 1, 1):
        delta_x = series_x[index + 1] - series_x[index]
        delta_y = series_y[index + 1] - series_y[index]
        slope = delta_y / delta_x
        return_x.append(series_x[index])
        return_y.append(slope)
    return return_x, return_y

def get_symmetric_derivative(series_x, series_y):
    if len(series_x) != len(series_y):
        raise ValueError(f"{series_x} did not have the same number of elements as {series_y}")
    return_x = list()
    return_y = list()
    # Remeber that the last element has no sucessor and we can therefore not get a derivative for it
    for index in range(1, len(series_x) - 1, 1):
        delta_x = series_x[index + 1] - series_x[index - 1]
        delta_y = series_y[index + 1] - series_y[index - 1]
        slope = delta_y / delta_x
        return_x.append(series_x[index])
        return_y.append(slope)
    return return_x, return_y

function = lambda x: x**3
analytic_derivative = lambda x: 3*(x**2)
step_size = 0.1
lower_bound = 0
upper_bound = 10
x_values = numpy.arange(lower_bound, upper_bound + step_size, step_size)
y_values = [function(x) for x in x_values]
y_values_analytic_derivative = [analytic_derivative(x) for x in x_values]
x_values_forward, y_values_forward = get_forward_derivative(x_values, y_values)
x_values_symmetric, y_values_symmetric = get_symmetric_derivative(x_values, y_values)

# Plot the analytic derivative to see if we are correct
matplotlib.pyplot.plot(x_values, y_values_analytic_derivative, label="$3 x^{2}$")
matplotlib.pyplot.plot(x_values_forward, y_values_forward, label="Forward")
matplotlib.pyplot.plot(x_values_symmetric, y_values_symmetric, label="Symmetric")

matplotlib.pyplot.title(f"Derivatives")
matplotlib.pyplot.xlabel("x")
matplotlib.pyplot.ylabel("y")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()
```
</details>

To round up this topic I would ask you to calculate the derivatives of the inter-spike-interval.
As the step-size $h$ use the index itself, by iterating over the recorded values.
Please, visualize them and their time dependence using the cell below.

In [ ]:
# Write your code here
# Feel free to copy from above what you need

<details>
<summary> Show suggested solution </summary>

```Python
import random
import csv
import pathlib
import matplotlib.pyplot

def get_forward_derivative(series_x, series_y):
    if len(series_x) != len(series_y):
        raise ValueError(f"{series_x} did not have the same number of elements as {series_y}")
    return_x = list()
    return_y = list()
    # Remeber that the last element has no sucessor and we can therefore not get a derivative for it
    for index in range(0, len(series_x) - 1, 1):
        delta_x = series_x[index + 1] - series_x[index]
        delta_y = series_y[index + 1] - series_y[index]
        slope = delta_y / delta_x
        return_x.append(series_x[index])
        return_y.append(slope)
    return return_x, return_y

def get_symmetric_derivative(series_x, series_y):
    if len(series_x) != len(series_y):
        raise ValueError(f"{series_x} did not have the same number of elements as {series_y}")
    return_x = list()
    return_y = list()
    # Remeber that the last element has no sucessor and we can therefore not get a derivative for it
    for index in range(1, len(series_x) - 1, 1):
        delta_x = series_x[index + 1] - series_x[index - 1]
        delta_y = series_y[index + 1] - series_y[index - 1]
        slope = delta_y / delta_x
        return_x.append(series_x[index])
        return_y.append(slope)
    return return_x, return_y

def get_spike_times_for_all_units(units_file_path):
    unit_spike_times = dict()
    with open(units_file_path, "r") as units_file:
        reader = csv.DictReader(units_file)
        for row in reader:
            unit_id = int(row["unitID"])
            if unit_id not in unit_spike_times.keys():
                unit_spike_times[unit_id] = list()
            spike_time = float(row["spikeTimes"])
            unit_spike_times[unit_id].append(spike_time)
    return unit_spike_times

def get_inter_spike_intervals(spike_times):
    differences = list()
    times = list()
    for index in range(0, len(spike_times) - 1):
        difference = spike_times[index + 1] - spike_times[index]
        differences.append(difference)
        average_time = (spike_times[index + 1] + spike_times[index]) / 2
        times.append(average_time)
    return differences, times

data_path = pathlib.Path("./data_neuron/")
units_file_path = data_path / "session_2023111501010_units.csv"

spikes_times_units = get_spike_times_for_all_units(units_file_path)
max_duration = 0
for unit_id in spikes_times_units.keys():
    max_time_unit = max(spikes_times_units[unit_id])
    max_duration = max(max_time_unit, max_duration)

for unit in spikes_times_units.keys():
    differences, times = get_inter_spike_intervals(spikes_times_units[unit])
    times, derivative = get_symmetric_derivative(times, differences)
    matplotlib.pyplot.plot(times, derivative, label=f"Unit {unit}")
matplotlib.pyplot.title("Derivatives")
matplotlib.pyplot.xlabel("Interval in seconds")
matplotlib.pyplot.ylabel("Change in interval length")
matplotlib.pyplot.legend()
matplotlib.pyplot.show()
```

</details>

## Rubber duck debugging

During this course, you were able to work with your partners.
Working on the course together, you were able to engage naturally in [pair programming](https://en.wikipedia.org/wiki/Pair_programming) and [code review](https://en.wikipedia.org/wiki/Code_review). 
These methods greatly improve your understanding of the problem and thereby your code.
It is unfortunately difficult to communicate the advantages of efficient work to superior and colleagues.
The preferred standard is to let two people on two desks solve in eight days, what two people on one desk could have solved in four.
Therefore, you will find yourself most probably working on your problems alone.
Luckily, not some of the benefits of working with a partner can be gained without one.

A good example of this is [rubber duck debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging).
It is especially useful for novice programmers as yourself or problems with many steps.
The general idea is that explaining something requires you to think more deeply about it,
therefore you can improve your understanding and by explaining.
So, you need to explain your code to figure out why it does not do what it is supposed to.
While a live human can contribute his or her own insights, a rubber duck is sufficient to gain your own.
So if you struggle with a problem take a rubber duck and explain the problem to the duck.
If you are lucky, you will realize that a small misunderstanding or mistake was the problem and fix it.

![Picture of a rubber duck in front of a monitor](https://upload.wikimedia.org/wikipedia/commons/d/d5/Rubber_duck_assisting_with_debugging.jpg)

## Possibilities 
After you are done with the exercise above take a look at the [matplotlib gallery](https://matplotlib.org/stable/gallery/index.html), [biopython](https://biopython.org/), [seaborn](https://seaborn.pydata.org/tutorial/introduction.html) and plan your future adventures with your rubber ducky.

If you have any questions feel free to ask them now.